## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-02-04-00-00 +0000,wsj,Modi Taps India’s Consumer Power in Trade Figh...,https://www.wsj.com/world/india/modi-taps-indi...
1,2026-01-02-03-45-02 +0000,nypost,Flash floods triggered by heavy rains in Afgha...,https://nypost.com/2026/01/01/world-news/flash...
2,2026-01-02-03-32-16 +0000,cbc,Several killed in Iran during violent protests...,https://www.cbc.ca/news/world/iran-protests-de...
3,2026-01-02-03-28-07 +0000,bbc,China's BYD set to overtake Tesla as world's t...,https://www.bbc.com/news/articles/cj9rjwpvmpzo...
4,2026-01-02-03-17-58 +0000,nypost,Jewish civil rights group demands answers from...,https://nypost.com/2026/01/01/us-news/jewish-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2464/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
88,new,43
89,year,39
7,trump,27
219,fire,17
220,swiss,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
125,2026-01-01-13-52-12 +0000,nypost,Horrific video shows the moment deadly New Yea...,https://nypost.com/2026/01/01/world-news/new-y...,158
141,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...,155
174,2026-01-01-07-42-14 +0000,nypost,"40 people dead, 100 hurt after fire rips throu...",https://nypost.com/2026/01/01/world-news/swiss...,152
45,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,150
31,2026-01-02-00-48-00 +0000,wsj,President Trump veered between business and pl...,https://www.wsj.com/politics/policy/golf-peace...,148


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
125,158,2026-01-01-13-52-12 +0000,nypost,Horrific video shows the moment deadly New Yea...,https://nypost.com/2026/01/01/world-news/new-y...
133,59,2026-01-01-12-48-00 +0000,wsj,President Trump delayed by a year tariff incre...,https://www.wsj.com/economy/trade/trump-rolls-...
42,41,2026-01-01-23-06-29 +0000,nypost,Mayor Zohran Mamdani issues sweeping executive...,https://nypost.com/2026/01/01/us-news/mayor-zo...
141,36,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...
96,26,2026-01-01-18-08-00 +0000,wsj,Iranian Protesters Killed as Unrest Turns Viol...,https://www.wsj.com/world/middle-east/iranian-...
114,25,2026-01-01-15-23-12 +0000,nypost,Prominent Jewish civil rights group demands Am...,https://nypost.com/2026/01/01/business/nationa...
112,25,2026-01-01-15-40-44 +0000,nyt,Trump Must Return Command of California Nation...,https://www.nytimes.com/2025/12/31/us/trump-na...
52,22,2026-01-01-22-24-25 +0000,nypost,Zelensky says peace deal is ‘90% ready’ in New...,https://nypost.com/2026/01/01/world-news/zelen...
59,21,2026-01-01-21-43-19 +0000,nypost,HHS audit for 2012 found $16M in ‘improper’ ch...,https://nypost.com/2026/01/01/us-news/hhs-audi...
55,21,2026-01-01-21-59-58 +0000,nypost,NYPD brought dozens of hearing and visually i...,https://nypost.com/2026/01/01/us-news/nypd-bro...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
